In [134]:
# Cell 1: Imports and Setup
# Import required libraries
import os
import json
import re
import glob
from pathlib import Path
from typing import List, Dict, Any, Tuple
import numpy as np

# LangChain imports
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import (
    Docx2txtLoader,
    UnstructuredWordDocumentLoader,
    PyPDFLoader,
    UnstructuredPDFLoader
)

# OCR related imports
import pytesseract
from pdf2image import convert_from_path
import io
from PIL import Image

print("Setup complete!")

# Cell 2: Configuration
# Configuration parameters
BASE_DIR = r"D:\Technical_projects\PSAI\raw_data\PSC"
YEAR_TO_PROCESS = "2014"  # Specify the year to process

# Derived paths
YEAR_DIR = os.path.join(BASE_DIR, YEAR_TO_PROCESS)
PROCESSED_DIR = os.path.join("processed_output", YEAR_TO_PROCESS)  # Where to store the processed full documents
CHUNKS_DIR = os.path.join("chunked_output", YEAR_TO_PROCESS)  # Where to store the chunked documents

# Chunking parameters
CHUNK_SIZE = 800  # Target size for chunks in characters
CHUNK_OVERLAP = 100  # Number of characters to overlap between chunks

# OCR configuration
# For Windows, you need to specify the path to Tesseract executable
# If using Linux, this can be commented out
TESSERACT_PATH = r"C:\Program Files\Tesseract-OCR\tesseract.exe"  # Update this path
pytesseract.pytesseract.tesseract_cmd = TESSERACT_PATH

# Create output directories if they don't exist
os.makedirs(PROCESSED_DIR, exist_ok=True)
os.makedirs(CHUNKS_DIR, exist_ok=True)

print(f"Configuration set for processing year: {YEAR_TO_PROCESS}")
print(f"Year directory: {YEAR_DIR}")
print(f"Processed documents will be stored in: {PROCESSED_DIR}")
print(f"Chunks will be stored in: {CHUNKS_DIR}")
print(f"Chunking parameters - Chunk size: {CHUNK_SIZE}, Overlap: {CHUNK_OVERLAP}")

# Cell 3: OCR Functions
def ocr_pdf(pdf_path):
    """
    Perform OCR on a PDF file using Tesseract.
    
    Args:
        pdf_path (str): Path to the PDF file
        
    Returns:
        str: Extracted text
    """
    try:
        print(f"Performing OCR on {os.path.basename(pdf_path)}...")
        
        # Convert PDF to images
        images = convert_from_path(pdf_path)
        
        # Process each page
        text_content = []
        
        for i, image in enumerate(images):
            print(f"  Processing page {i+1}/{len(images)}")
            
            # Perform OCR
            text = pytesseract.image_to_string(image, lang='eng')
            text_content.append(text)
        
        # Combine all pages
        full_text = "\n\n".join(text_content)
        
        print(f"  OCR completed. Extracted {len(full_text)} characters.")
        return full_text
        
    except Exception as e:
        print(f"Error performing OCR on {pdf_path}: {e}")
        import traceback
        traceback.print_exc()
        return f"OCR Error: {str(e)}"

def check_pdf_has_text(pdf_path):
    """
    Check if a PDF has extractable text or needs OCR.
    
    Args:
        pdf_path (str): Path to the PDF file
        
    Returns:
        bool: True if the PDF has extractable text, False if it needs OCR
    """
    try:
        # Try to extract text using PyPDF2 first
        from PyPDF2 import PdfReader
        reader = PdfReader(pdf_path)
        
        # Sample the first page
        page = reader.pages[0]
        text = page.extract_text()
        
        # Check if there's meaningful text (more than just spaces or a few characters)
        # This is a simple heuristic - might need tuning
        if text and len(text.strip()) > 100:
            return True
            
        # If the text is too short, try another page if available
        if len(reader.pages) > 1:
            page = reader.pages[1]
            text = page.extract_text()
            if text and len(text.strip()) > 100:
                return True
        
        # If we got here, PDF likely needs OCR
        return False
        
    except Exception as e:
        print(f"Error checking PDF text extractability: {e}")
        # If we couldn't check, assume OCR is needed
        return False

# Cell 4: Document Processing Functions
def extract_metadata_from_filename(filename):
    """
    Extract date and other metadata from filename.
    
    Args:
        filename (str): Original filename
        
    Returns:
        dict: Metadata dictionary
    """
    metadata = {
        "title": "",
        "date": "",
        "author": "Phyllis Schlafly",
        "subjects": [],
        "page_number": 1,
        "source_file": os.path.basename(filename),
        "doc_type": "Phyllis Schlafly Column"
    }
    
    # Extract date from filename
    date_match = re.search(r'PSC_(\d{4})_(\d{2})_(\d{2})', filename)
    
    if date_match:
        year, month, day = date_match.groups()
        # Convert month number to month name
        month_names = [
            "January", "February", "March", "April", "May", "June",
            "July", "August", "September", "October", "November", "December"
        ]
        try:
            month_name = month_names[int(month) - 1]
            metadata["date"] = f"{month_name} {int(day)}, {year}"
        except:
            # Fallback if date parsing fails
            metadata["date"] = f"{year}-{month}-{day}"
    
    return metadata

def get_processed_files(processed_dir):
    """Get list of already processed files"""
    return [os.path.basename(f) for f in glob.glob(os.path.join(processed_dir, "*.json"))]

def extract_text_from_file(file_path):
    """
    Extract text from a file based on its extension, with OCR support.
    
    Args:
        file_path (str): Path to the file
        
    Returns:
        str: Extracted text content
    """
    file_ext = os.path.splitext(file_path)[1].lower()
    
    try:
        if file_ext == '.docx':
            # Use Docx2txtLoader from LangChain
            loader = Docx2txtLoader(file_path)
            documents = loader.load()
            return "\n\n".join([doc.page_content for doc in documents])
            
        elif file_ext == '.doc':
            # Use UnstructuredWordDocumentLoader from LangChain
            try:
                loader = UnstructuredWordDocumentLoader(file_path)
                documents = loader.load()
                return "\n\n".join([doc.page_content for doc in documents])
            except Exception as e:
                print(f"Error with UnstructuredWordDocumentLoader: {e}")
                # Alternative method if the first fails
                import textract
                return textract.process(file_path).decode('utf-8')
                
        elif file_ext == '.pdf':
            # Check if PDF has extractable text or needs OCR
            if check_pdf_has_text(file_path):
                print(f"PDF has extractable text: {os.path.basename(file_path)}")
                # Try PyPDFLoader first
                try:
                    loader = PyPDFLoader(file_path)
                    documents = loader.load()
                    content = "\n\n".join([doc.page_content for doc in documents])
                    # If content is too short, likely needs OCR anyway
                    if len(content.strip()) < 200:
                        print(f"Extracted text too short ({len(content.strip())} chars), falling back to OCR")
                        return ocr_pdf(file_path)
                    return content
                except Exception as e:
                    print(f"Error with PyPDFLoader: {e}")
                    # Fall back to UnstructuredPDFLoader
                    try:
                        loader = UnstructuredPDFLoader(file_path)
                        documents = loader.load()
                        content = "\n\n".join([doc.page_content for doc in documents])
                        if len(content.strip()) < 200:
                            print(f"Extracted text too short ({len(content.strip())} chars), falling back to OCR")
                            return ocr_pdf(file_path)
                        return content
                    except Exception as e2:
                        print(f"Error with UnstructuredPDFLoader: {e2}")
                        # If both fail, use OCR
                        return ocr_pdf(file_path)
            else:
                print(f"PDF needs OCR: {os.path.basename(file_path)}")
                # Perform OCR
                return ocr_pdf(file_path)
        else:
            return f"Unsupported file format: {file_ext}"
    except Exception as e:
        print(f"Error extracting text from {file_path}: {e}")
        return f"Error extracting text: {str(e)}"

def process_documents_from_raw():
    """Process raw documents from year directory and save to processed_dir"""
    # Get all DOCX, DOC, and PDF files
    docx_files = glob.glob(os.path.join(YEAR_DIR, "*.docx"))
    doc_files = glob.glob(os.path.join(YEAR_DIR, "*.doc"))
    pdf_files = glob.glob(os.path.join(YEAR_DIR, "*.pdf"))
    
    print(f"Found {len(docx_files)} DOCX files, {len(doc_files)} DOC files, and {len(pdf_files)} PDF files")
    
    # Group files by base name
    file_groups = {}
    for f in docx_files + doc_files + pdf_files:
        base_name = os.path.splitext(os.path.basename(f))[0]
        if base_name not in file_groups:
            file_groups[base_name] = []
        file_groups[base_name].append(f)
    
    print(f"Grouped into {len(file_groups)} unique documents")
    
    # Process each group, prioritizing DOCX > DOC > PDF
    processed_count = 0
    skipped_count = 0
    processed_files = get_processed_files(PROCESSED_DIR)
    
    for base_name, files in file_groups.items():
        # Check if this file has already been processed
        if any(base_name in pf for pf in processed_files):
            print(f"Skipping {base_name} - already processed")
            skipped_count += 1
            continue
        
        # Sort by priority (.docx first, then .doc, then .pdf)
        extensions = [os.path.splitext(f)[1].lower() for f in files]
        priorities = []
        for ext in extensions:
            if ext == ".docx":
                priorities.append(0)
            elif ext == ".doc":
                priorities.append(1)
            elif ext == ".pdf":
                priorities.append(2)
            else:
                priorities.append(3)
        
        # Get the file with highest priority (lowest number)
        best_file = files[priorities.index(min(priorities))]
        print(f"Processing {os.path.basename(best_file)}")
        
        # Extract metadata
        metadata = extract_metadata_from_filename(best_file)
        
        # Extract text
        content = extract_text_from_file(best_file)
        
        # Create the document object
        document = {
            "text": content,
            "metadata": metadata
        }
        
        # Save the processed document
        output_file = os.path.join(PROCESSED_DIR, f"{base_name}.json")
        with open(output_file, 'w', encoding='utf-8') as f:
            json.dump(document, f, ensure_ascii=False, indent=2)
        
        processed_count += 1
    
    print(f"Processed {processed_count} new documents, skipped {skipped_count} already processed")
    return processed_count

# Cell 5: Document Loading Functions
def load_processed_documents() -> List[Dict[str, Any]]:
    """
    Load all processed documents from the processed directory.
    
    Returns:
        List[Dict[str, Any]]: List of processed document dictionaries
    """
    documents = []
    
    # Get all JSON files in the processed directory
    json_files = glob.glob(os.path.join(PROCESSED_DIR, "*.json"))
    
    print(f"Found {len(json_files)} processed documents")
    
    for json_file in json_files:
        try:
            with open(json_file, 'r', encoding='utf-8') as f:
                document = json.load(f)
                documents.append(document)
        except Exception as e:
            print(f"Error loading document {json_file}: {e}")
    
    return documents

# Cell 6: Chunking Function with LangChain
def chunk_document(document: Dict[str, Any]) -> List[Dict[str, Any]]:
    """
    Split a document into chunks using LangChain's RecursiveCharacterTextSplitter.
    
    Args:
        document (Dict[str, Any]): The document to chunk
        
    Returns:
        List[Dict[str, Any]]: List of document chunks
    """
    text = document.get("text", "")
    metadata = document.get("metadata", {})
    
    # Handle empty documents
    if not text:
        print(f"Warning: Empty text in document {metadata.get('source_file', 'unknown')}")
        return []
    
    # Create a text splitter
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=CHUNK_SIZE,
        chunk_overlap=CHUNK_OVERLAP,
        length_function=len,
        separators=["\n\n", "\n", ". ", " ", ""]
    )
    
    # Split the text into chunks
    chunk_texts = text_splitter.split_text(text)
    
    # Create document chunks with metadata
    chunks = []
    for i, chunk_text in enumerate(chunk_texts):
        # Create a copy of the metadata
        chunk_metadata = metadata.copy()
        
        # Add chunk information to metadata
        chunk_metadata["chunk_id"] = i + 1
        chunk_metadata["total_chunks"] = len(chunk_texts)
        
        # Create chunk
        chunk = {
            "text": chunk_text,
            "metadata": chunk_metadata
        }
        
        chunks.append(chunk)
    
    return chunks

# Cell 7: Process and Chunk Documents
def process_chunks_for_documents() -> Tuple[int, int]:
    """
    Process all documents and create chunks.
    
    Returns:
        Tuple[int, int]: (Number of documents processed, Number of chunks created)
    """
    # Load the processed documents
    documents = load_processed_documents()
    
    total_chunks = 0
    total_docs = len(documents)
    
    # Check if we have any documents to process
    if total_docs == 0:
        print("No documents found to chunk. Please run the document processing first.")
        return 0, 0
    
    # Process each document
    for doc_index, document in enumerate(documents):
        source_file = document["metadata"].get("source_file", f"document_{doc_index}")
        print(f"Processing document {doc_index+1}/{total_docs}: {source_file}")
        
        # Create chunks for this document
        chunks = chunk_document(document)
        
        # Save each chunk as a separate JSON file
        base_name = os.path.splitext(source_file)[0]
        
        for i, chunk in enumerate(chunks):
            chunk_file = os.path.join(
                CHUNKS_DIR, 
                f"{base_name}_chunk_{i+1}of{len(chunks)}.json"
            )
            
            with open(chunk_file, 'w', encoding='utf-8') as f:
                json.dump(chunk, f, ensure_ascii=False, indent=2)
        
        print(f"  Created {len(chunks)} chunks")
        total_chunks += len(chunks)
    
    # Save a summary file
    summary_data = {
        "year": YEAR_TO_PROCESS,
        "documents_processed": total_docs,
        "chunks_created": total_chunks,
        "chunk_params": {
            "chunk_size": CHUNK_SIZE,
            "chunk_overlap": CHUNK_OVERLAP
        }
    }
    
    summary_file = os.path.join(CHUNKS_DIR, f"chunks_summary.json")
    with open(summary_file, 'w', encoding='utf-8') as f:
        json.dump(summary_data, f, ensure_ascii=False, indent=2)
    
    return total_docs, total_chunks

# Cell 8: Chunk Analysis
def analyze_chunks() -> Dict[str, Any]:
    """
    Analyze the created chunks and generate statistics.
    
    Returns:
        Dict[str, Any]: Analysis results
    """
    # Get all chunk files
    chunk_pattern = os.path.join(CHUNKS_DIR, "*_chunk_*.json")
    chunk_files = glob.glob(chunk_pattern)
    
    # If no chunks found, return empty analysis
    if not chunk_files:
        print("No chunks found to analyze.")
        return {
            "year": YEAR_TO_PROCESS,
            "total_chunks": 0,
            "chunk_size_stats": {
                "min": 0, "max": 0, "mean": 0, "median": 0, "std": 0
            },
            "chunks_per_doc_stats": {
                "min": 0, "max": 0, "mean": 0, "median": 0, "total_docs": 0
            }
        }
    
    # Collect statistics
    chunk_sizes = []
    chunks_per_doc = {}
    
    for chunk_file in chunk_files:
        try:
            with open(chunk_file, 'r', encoding='utf-8') as f:
                chunk = json.load(f)
                
                # Get chunk size in characters
                chunk_size = len(chunk["text"])
                chunk_sizes.append(chunk_size)
                
                # Track chunks per document
                source_file = chunk["metadata"].get("source_file", "unknown")
                if source_file not in chunks_per_doc:
                    chunks_per_doc[source_file] = 0
                chunks_per_doc[source_file] += 1
                
        except Exception as e:
            print(f"Error analyzing chunk {chunk_file}: {e}")
    
    # Calculate statistics
    analysis = {
        "year": YEAR_TO_PROCESS,
        "total_chunks": len(chunk_sizes),
        "chunk_size_stats": {
            "min": min(chunk_sizes) if chunk_sizes else 0,
            "max": max(chunk_sizes) if chunk_sizes else 0,
            "mean": float(np.mean(chunk_sizes)) if chunk_sizes else 0,
            "median": float(np.median(chunk_sizes)) if chunk_sizes else 0,
            "std": float(np.std(chunk_sizes)) if chunk_sizes else 0
        },
        "chunks_per_doc_stats": {
            "min": min(chunks_per_doc.values()) if chunks_per_doc else 0,
            "max": max(chunks_per_doc.values()) if chunks_per_doc else 0,
            "mean": float(np.mean(list(chunks_per_doc.values()))) if chunks_per_doc else 0,
            "median": float(np.median(list(chunks_per_doc.values()))) if chunks_per_doc else 0,
            "total_docs": len(chunks_per_doc)
        }
    }
    
    # Save analysis
    analysis_file = os.path.join(CHUNKS_DIR, f"chunk_analysis.json")
    with open(analysis_file, 'w', encoding='utf-8') as f:
        json.dump(analysis, f, ensure_ascii=False, indent=2)
    
    return analysis

Setup complete!
Configuration set for processing year: 2014
Year directory: D:\Technical_projects\PSAI\raw_data\PSC\2014
Processed documents will be stored in: processed_output\2014
Chunks will be stored in: chunked_output\2014
Chunking parameters - Chunk size: 800, Overlap: 100


In [135]:
# Cell 9: Main Execution
try:
    # First, process any raw documents that need processing
    new_docs_processed = process_documents_from_raw()
    print(f"Processed {new_docs_processed} new documents")

    # Now, chunk all processed documents
    total_docs, total_chunks = process_chunks_for_documents()
    print(f"\nChunked {total_docs} documents into {total_chunks} chunks for year {YEAR_TO_PROCESS}")

    # Only analyze if we have chunks
    if total_chunks > 0:
        # Analyze chunks
        analysis = analyze_chunks()
        print("\nChunk Analysis:")
        print(f"Total chunks: {analysis['total_chunks']}")
        print(f"Average chunk size: {analysis['chunk_size_stats']['mean']:.1f} characters")
        print(f"Average chunks per document: {analysis['chunks_per_doc_stats']['mean']:.1f}")
    else:
        print("\nNo chunks to analyze.")
except Exception as e:
    print(f"\nAn error occurred: {e}")
    import traceback
    traceback.print_exc()
    print("\nTroubleshooting tips:")
    print("1. Make sure all required libraries are installed")
    print("2. Verify folder paths exist and are accessible")
    print("3. Check that document files are readable")
    print("4. Ensure Tesseract OCR is properly installed and the path is correct")

Found 28 DOCX files, 22 DOC files, and 1 PDF files
Grouped into 51 unique documents
Processing PSC_2014_01_14.docx
Processing PSC_2014_01_28.docx
Processing PSC_2014_02_11.docx
Processing PSC_2014_02_18.docx
Processing PSC_2014_03_04.docx
Processing PSC_2014_03_11.docx
Processing PSC_2014_03_25.docx
Processing PSC_2014_04_01.docx
Processing PSC_2014_04_08.docx
Processing PSC_2014_04_15.docx
Processing PSC_2014_04_22.docx
Processing PSC_2014_04_29.docx
Processing PSC_2014_05_06.docx
Processing PSC_2014_05_13.docx
Processing PSC_2014_05_20.docx
Processing PSC_2014_05_27.docx
Processing PSC_2014_06_03.docx
Processing PSC_2014_06_10.docx
Processing PSC_2014_06_17.docx
Processing PSC_2014_06_24.docx
Processing PSC_2014_07_02.docx
Processing PSC_2014_07_09.docx
Processing PSC_2014_07_16.docx
Processing PSC_2014_07_22.docx
Processing PSC_2014_08_05.docx
Processing PSC_2014_08_12.docx
Processing PSC_2017_07_29.docx
Processing ~$c042914.docx
Error extracting text from D:\Technical_projects\PSAI

In [136]:
# Cell 1: Imports and Setup
import os
import json
import glob
from pathlib import Path
import pandas as pd

print("Setup complete!")

# Cell 2: Configuration
# Base directories
CHUNKS_BASE_DIR = "chunked_output"  # Base directory for chunked files
CONSOLIDATED_DIR = "consolidated_output"  # Directory to save consolidated files

# Create output directory if it doesn't exist
os.makedirs(CONSOLIDATED_DIR, exist_ok=True)

print(f"Consolidated files will be saved to: {CONSOLIDATED_DIR}")

# Cell 3: Consolidation Function
def consolidate_year_chunks(year, input_dir=None, output_dir=None):
    """
    Consolidate all JSON chunk files for a specific year into a single JSON file.
    
    Args:
        year (str): The year to process
        input_dir (str, optional): Base directory for chunked files. Defaults to "chunked_output".
        output_dir (str, optional): Directory to save consolidated files. Defaults to "consolidated_output".
    
    Returns:
        tuple: (Number of chunks processed, Total file size)
    """
    # Set up directories
    if input_dir is None:
        input_dir = CHUNKS_BASE_DIR
    
    if output_dir is None:
        output_dir = CONSOLIDATED_DIR
    
    year_input_dir = os.path.join(input_dir, year)
    
    # Check if the year directory exists
    if not os.path.exists(year_input_dir):
        print(f"Directory not found for year {year}: {year_input_dir}")
        return 0, 0
    
    # Get all JSON chunk files for this year
    chunk_pattern = os.path.join(year_input_dir, "*_chunk_*.json")
    chunk_files = glob.glob(chunk_pattern)
    
    if not chunk_files:
        print(f"No chunk files found for year {year} in {year_input_dir}")
        return 0, 0
    
    print(f"Found {len(chunk_files)} chunk files for year {year}")
    
    # Load all chunks
    all_chunks = []
    for chunk_file in chunk_files:
        try:
            with open(chunk_file, 'r', encoding='utf-8') as f:
                chunk = json.load(f)
                all_chunks.append(chunk)
        except Exception as e:
            print(f"Error loading {chunk_file}: {e}")
    
    # Sort chunks by date (if available) and then by source_file and chunk_id
    def chunk_sort_key(chunk):
        metadata = chunk.get("metadata", {})
        date = metadata.get("date", "")
        source_file = metadata.get("source_file", "")
        chunk_id = metadata.get("chunk_id", 0)
        return (date, source_file, chunk_id)
    
    all_chunks.sort(key=chunk_sort_key)
    
    # Create the consolidated JSON file
    output_file = os.path.join(output_dir, f"psc_{year}_all_chunks.json")
    
    # Create a JSON object with metadata and the chunks
    consolidated_data = {
        "year": year,
        "chunk_count": len(all_chunks),
        "chunks": all_chunks
    }
    
    # Save the consolidated file
    with open(output_file, 'w', encoding='utf-8') as f:
        json.dump(consolidated_data, f, ensure_ascii=False, indent=2)
    
    # Get file size
    file_size = os.path.getsize(output_file)
    file_size_mb = file_size / (1024 * 1024)
    
    print(f"Created consolidated file: {output_file}")
    print(f"File contains {len(all_chunks)} chunks")
    print(f"File size: {file_size_mb:.2f} MB")
    
    return len(all_chunks), file_size


Setup complete!
Consolidated files will be saved to: consolidated_output


In [ ]:

# # Cell 4: Process a Single Year
# # Change this to the year you want to process
# YEAR_TO_PROCESS = "1973"

# # Process the year
# chunks, size = consolidate_year_chunks(YEAR_TO_PROCESS)

# if chunks > 0:
#     size_mb = size / (1024 * 1024)
#     print(f"\nSuccessfully consolidated {chunks} chunks for year {YEAR_TO_PROCESS}")
#     print(f"File size: {size_mb:.2f} MB")
# else:
#     print(f"\nNo chunks were processed for year {YEAR_TO_PROCESS}")


In [137]:

# Cell 5: Process All Years
def get_available_years():
    """Get a list of all available years in the chunks directory"""
    year_dirs = glob.glob(os.path.join(CHUNKS_BASE_DIR, "*"))
    years = []
    
    for year_dir in year_dirs:
        if os.path.isdir(year_dir):
            year = os.path.basename(year_dir)
            if year.isdigit():  # Make sure it's a year
                years.append(year)
    
    return sorted(years)

available_years = get_available_years()
print(f"Found {len(available_years)} years to process: {', '.join(available_years)}")

# Cell 6: Process All Years
def process_all_years(years):
    """Process a list of years"""
    total_chunks = 0
    total_size = 0
    
    results = []
    
    for year in years:
        print(f"\nProcessing year {year}...")
        
        chunks, size = consolidate_year_chunks(year)
        
        size_mb = size / (1024 * 1024)
        results.append({
            "year": year,
            "chunks": chunks,
            "size_mb": size_mb
        })
        
        total_chunks += chunks
        total_size += size
    
    total_size_mb = total_size / (1024 * 1024)
    print(f"\nConsolidation complete! Processed {total_chunks} chunks across {len(years)} years")
    print(f"Total size of all consolidated files: {total_size_mb:.2f} MB")
    
    return results

# Process all available years
results = process_all_years(available_years)

# Cell 7: Display Results Table
# Convert results to DataFrame for nice display
df = pd.DataFrame(results)
df = df.sort_values("year")
df["size_mb"] = df["size_mb"].round(2)

# Calculate total
total_row = {
    "year": "TOTAL",
    "chunks": df["chunks"].sum(),
    "size_mb": df["size_mb"].sum().round(2)
}
df = df.append(total_row, ignore_index=True)

print("Consolidation Results:")
print(df)

# Cell 8: Verify Consolidated Files
def verify_consolidated_files(years):
    """Verify that all expected consolidated files exist and are valid JSON"""
    missing = []
    invalid = []
    verified = []
    
    for year in years:
        file_path = os.path.join(CONSOLIDATED_DIR, f"psc_{year}_all_chunks.json")
        
        if not os.path.exists(file_path):
            missing.append(year)
            continue
        
        try:
            with open(file_path, 'r', encoding='utf-8') as f:
                data = json.load(f)
                
                # Basic validation
                if "chunks" not in data or not isinstance(data["chunks"], list):
                    invalid.append(year)
                    continue
                
                chunk_count = len(data["chunks"])
                file_size = os.path.getsize(file_path) / (1024 * 1024)
                
                verified.append({
                    "year": year,
                    "chunks": chunk_count,
                    "size_mb": round(file_size, 2)
                })
        except Exception as e:
            print(f"Error verifying {file_path}: {e}")
            invalid.append(year)
    
    print("\nVerification Results:")
    print(f"Verified: {len(verified)} files")
    
    if missing:
        print(f"Missing: {len(missing)} files - {', '.join(missing)}")
    
    if invalid:
        print(f"Invalid: {len(invalid)} files - {', '.join(invalid)}")
    
    return verified, missing, invalid

# Verify the consolidated files
verified, missing, invalid = verify_consolidated_files(available_years)

# Display verified files as a DataFrame
if verified:
    verify_df = pd.DataFrame(verified)
    verify_df = verify_df.sort_values("year")
    print("\nVerified Files:")
    print(verify_df)

Found 42 years to process: 1973, 1974, 1975, 1976, 1977, 1978, 1979, 1980, 1981, 1982, 1983, 1984, 1985, 1986, 1987, 1988, 1989, 1990, 1991, 1992, 1993, 1994, 1995, 1996, 1997, 1998, 1999, 2000, 2001, 2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014

Processing year 1973...
Found 218 chunk files for year 1973
Created consolidated file: consolidated_output\psc_1973_all_chunks.json
File contains 218 chunks
File size: 0.20 MB

Processing year 1974...
Found 1043 chunk files for year 1974
Created consolidated file: consolidated_output\psc_1974_all_chunks.json
File contains 1043 chunks
File size: 0.92 MB

Processing year 1975...
Found 587 chunk files for year 1975
Created consolidated file: consolidated_output\psc_1975_all_chunks.json
File contains 587 chunks
File size: 0.53 MB

Processing year 1976...
Found 623 chunk files for year 1976
Created consolidated file: consolidated_output\psc_1976_all_chunks.json
File contains 623 chunks
File size: 0.57 MB

Processing 

AttributeError: 'DataFrame' object has no attribute 'append'

In [4]:
# Cell 1: Install required dependencies
# Run this cell first to ensure all dependencies are installed
import sys
import subprocess

def install_dependencies():
    """Install required packages if they're not already installed"""
    required_packages = ['pandas', 'openpyxl']
    
    for package in required_packages:
        try:
            __import__(package)
            print(f"✓ {package} is already installed")
        except ImportError:
            print(f"Installing {package}...")
            subprocess.check_call([sys.executable, "-m", "pip", "install", package])
            print(f"✓ {package} has been installed")

# Install required dependencies
install_dependencies()

# Cell 2: Import necessary libraries
import json
import os
import glob
import re
import pandas as pd
import chardet  # For detecting file encoding

# Cell 3: Define the paths to your files
# Update these paths to match your environment
CONSOLIDATED_DIR = r"D:\Technical_projects\PSAI\code\consolidated_output"  # Directory with your consolidated JSON files
METADATA_FILES = [
    r"D:\Technical_projects\PSAI\raw_data\PSC\000_LIMBFILES\1970s.xlsx",
    r"D:\Technical_projects\PSAI\raw_data\PSC\000_LIMBFILES\80s.xlsx",
    r"D:\Technical_projects\PSAI\raw_data\PSC\000_LIMBFILES\90s.csv",
    r"D:\Technical_projects\PSAI\raw_data\PSC\000_LIMBFILES\00s.csv"
]

# Cell 4: Define function to detect file encoding
def detect_encoding(file_path):
    """
    Detect the encoding of a file using chardet
    
    Args:
        file_path (str): Path to the file
        
    Returns:
        str: Detected encoding
    """
    try:
        # Install chardet if not present
        try:
            import chardet
        except ImportError:
            print("Installing chardet package...")
            subprocess.check_call([sys.executable, "-m", "pip", "install", "chardet"])
            import chardet
            print("chardet installed successfully")
        
        # Read a sample of the file to detect encoding
        with open(file_path, 'rb') as f:
            # Read up to 1MB of the file
            raw_data = f.read(1024 * 1024)
        
        result = chardet.detect(raw_data)
        encoding = result['encoding']
        confidence = result['confidence']
        
        print(f"Detected encoding for {os.path.basename(file_path)}: {encoding} (confidence: {confidence:.2f})")
        
        # If encoding is None or confidence is low, use a common fallback
        if not encoding or confidence < 0.7:
            print(f"Low confidence in encoding detection, trying common encodings")
            # Try common encodings
            for enc in ['utf-8', 'latin1', 'iso-8859-1', 'cp1252', 'utf-16']:
                try:
                    with open(file_path, 'r', encoding=enc) as f:
                        # Try to read a few lines
                        for _ in range(5):
                            f.readline()
                    print(f"Successfully read with {enc} encoding")
                    return enc
                except UnicodeDecodeError:
                    continue
            
            # If all else fails, use latin1 which rarely fails
            print("Falling back to latin1 encoding")
            return 'latin1'
        
        return encoding
    
    except Exception as e:
        print(f"Error detecting encoding: {e}")
        print("Falling back to latin1 encoding")
        return 'latin1'  # Fallback to latin1 which rarely fails

# Cell 5: Define functions for loading metadata and updating JSON files
def load_metadata_from_excel_csv(file_paths):
    """
    Load metadata from Excel and CSV files.
    
    Args:
        file_paths (list): List of paths to Excel and CSV files
        
    Returns:
        dict: Dictionary mapping filenames to their metadata (title and subjects)
    """
    metadata_map = {}
    
    for file_path in file_paths:
        print(f"Loading metadata from: {file_path}")
        
        try:
            # Load file based on extension
            if file_path.endswith('.xlsx') or file_path.endswith('.xls'):
                try:
                    # First try with default engine
                    df = pd.read_excel(file_path)
                except ImportError as e:
                    print(f"Error: {e}")
                    print("Installing openpyxl...")
                    subprocess.check_call([sys.executable, "-m", "pip", "install", "openpyxl"])
                    try:
                        df = pd.read_excel(file_path)
                    except Exception as e2:
                        print(f"Error loading Excel file after installing openpyxl: {e2}")
                        print("Converting Excel to CSV format...")
                        # Try to use alternative method
                        try:
                            import subprocess
                            temp_csv = file_path + ".temp.csv"
                            cmd = [sys.executable, "-c", 
                                f"import pandas as pd; pd.read_excel('{file_path}').to_csv('{temp_csv}', index=False)"]
                            subprocess.check_call(cmd)
                            
                            # Detect encoding for the CSV
                            encoding = detect_encoding(temp_csv)
                            df = pd.read_csv(temp_csv, encoding=encoding)
                            
                            # Clean up temporary file
                            os.remove(temp_csv)
                        except Exception as e3:
                            print(f"Failed to convert Excel to CSV: {e3}")
                            print(f"Skipping file {file_path}")
                            continue
                            
            elif file_path.endswith('.csv'):
                # Detect the encoding of the CSV file
                encoding = detect_encoding(file_path)
                
                # Try to read with detected encoding
                try:
                    df = pd.read_csv(file_path, encoding=encoding)
                except Exception as e:
                    print(f"Error reading CSV with detected encoding: {e}")
                    # Try alternative encodings if the detected one fails
                    for alt_encoding in ['latin1', 'cp1252', 'iso-8859-1', 'utf-16']:
                        if alt_encoding != encoding:
                            try:
                                print(f"Trying alternate encoding: {alt_encoding}")
                                df = pd.read_csv(file_path, encoding=alt_encoding)
                                print(f"Successfully read with {alt_encoding}")
                                break
                            except Exception:
                                continue
                    else:
                        print(f"Failed to read CSV with any encoding. Skipping file: {file_path}")
                        continue
            else:
                print(f"Unsupported file format: {file_path}")
                continue
            
            # Check if we have the necessary columns
            required_columns = ['file name', 'title']
            missing_columns = [col for col in required_columns if col not in df.columns]
            
            if missing_columns:
                print(f"Warning: Missing required columns in {file_path}: {missing_columns}")
                # Try to find alternative column names
                column_mapping = {}
                for req_col in missing_columns:
                    # Look for similar columns
                    for col in df.columns:
                        if req_col.lower() in col.lower():
                            column_mapping[col] = req_col
                            print(f"Using '{col}' as '{req_col}'")
                            break
                
                # Rename columns if alternatives were found
                if column_mapping:
                    df = df.rename(columns=column_mapping)
                
                # Check again for required columns
                still_missing = [col for col in required_columns if col not in df.columns]
                if still_missing:
                    print(f"Still missing required columns: {still_missing}. Skipping file.")
                    continue
            
            # Print column information
            print(f"Columns in file: {list(df.columns)}")
            
            # Process each row
            for i, row in df.iterrows():
                # Get filename, title, and subject
                try:
                    # Try to handle potential NaN or missing values
                    file_name = row.get('file name')
                    if pd.isna(file_name):
                        continue
                        
                    title = row.get('title')
                    if pd.isna(title):
                        continue
                    
                    # Look for subject column with various possible names
                    subject = None
                    for subject_col in ['subject', 'subjects', 'Subject', 'Subjects', 'SUBJECT', 'SUBJECTS']:
                        if subject_col in row:
                            subject = row.get(subject_col)
                            break
                    
                    # Skip if any essential field is missing or NaN
                    if not file_name or not title or pd.isna(file_name) or pd.isna(title):
                        continue
                    
                    # Convert to string and clean up
                    file_name = str(file_name).strip().strip('"\'')
                    title = str(title).strip()
                    
                    # Extract the base filename without extension
                    base_name = os.path.splitext(file_name)[0]
                    
                    # Split subjects if they contain semicolons or commas
                    subject_list = []
                    if subject and not pd.isna(subject):
                        # Convert to string in case it's a different type
                        subject = str(subject)
                        # Handle both semicolon and comma separators
                        if ';' in subject:
                            subject_list = [s.strip() for s in subject.split(';')]
                        elif ',' in subject:
                            subject_list = [s.strip() for s in subject.split(',')]
                        else:
                            subject_list = [subject.strip()]
                    
                    # Store in the map
                    metadata_map[base_name] = {
                        'title': title,
                        'subjects': subject_list
                    }
                    
                except Exception as e:
                    print(f"Error processing row {i}: {e}")
                    continue
        except Exception as e:
            print(f"Error processing file {file_path}: {e}")
            continue
    
    print(f"Loaded metadata for {len(metadata_map)} documents")
    
    # Print a sample of the metadata for verification
    print("\nSample metadata entries:")
    sample_keys = list(metadata_map.keys())[:5]  # First 5 keys
    for key in sample_keys:
        print(f"  {key}: {metadata_map[key]}")
    
    return metadata_map

def update_json_with_metadata(json_file_path, metadata_map):
    """
    Update a consolidated JSON file with titles and subjects from the metadata map.
    
    Args:
        json_file_path (str): Path to the consolidated JSON file
        metadata_map (dict): Dictionary mapping filenames to metadata
        
    Returns:
        int: Number of chunks updated
    """
    print(f"Updating file: {json_file_path}")
    
    # Load the JSON file
    with open(json_file_path, 'r', encoding='utf-8') as f:
        data = json.load(f)
    
    # Count of updated chunks
    updated_count = 0
    total_chunks = len(data.get("chunks", []))
    
    # Track which files were matched and which weren't
    matched_files = set()
    unmatched_files = set()
    
    # First pass: collect all unique source files
    source_files = set()
    source_file_to_chunks = {}
    
    for i, chunk in enumerate(data.get("chunks", [])):
        metadata = chunk.get("metadata", {})
        source_file = metadata.get("source_file", "")
        
        if source_file:
            source_files.add(source_file)
            
            # Group chunks by source file
            if source_file not in source_file_to_chunks:
                source_file_to_chunks[source_file] = []
            source_file_to_chunks[source_file].append(i)
    
    print(f"Found {len(source_files)} unique source files in {total_chunks} chunks")
    
    # Print a sample of source files for debugging
    print("Sample source files:")
    sample_source_files = list(source_files)[:5]
    for source_file in sample_source_files:
        print(f"  - {source_file}")
    
    # Second pass: match source files to metadata
    source_file_metadata = {}
    
    for source_file in source_files:
        # Extract base name without extension
        base_name = os.path.splitext(source_file)[0]
        
        # Also try with .pdf extension
        pdf_base_name = base_name + ".pdf"
        pdf_base_name_without_ext = os.path.splitext(pdf_base_name)[0]
        
        # Check for metadata match
        metadata_entry = None
        matched_key = None
        
        # Try multiple possible matching strategies
        possible_keys = [
            base_name,                                   # Direct match
            pdf_base_name_without_ext,                   # Match with PDF extension
            f"PSC_{base_name}" if not base_name.startswith("PSC_") else None,  # With PSC_ prefix
            base_name[4:] if base_name.startswith("PSC_") else None,           # Without PSC_ prefix
            base_name.replace("PSR_", "PSC_") if base_name.startswith("PSR_") else None,  # PSR to PSC
            base_name.replace("PSC_", "PSR_") if base_name.startswith("PSC_") else None   # PSC to PSR
        ]
        
        # Try all possible keys
        for key in possible_keys:
            if key and key in metadata_map:
                metadata_entry = metadata_map[key]
                matched_key = key
                break
        
        # Try matching by date pattern if still not found
        if not metadata_entry:
            # Extract date pattern (YYYY_MM_DD or similar)
            date_patterns = [
                re.search(r'(\d{4})[_-](\d{2})[_-](\d{2})', base_name),  # YYYY-MM-DD
                re.search(r'(\d{4})[_-](\d{2})', base_name)               # YYYY-MM
            ]
            
            for date_match in date_patterns:
                if date_match:
                    date_pattern = date_match.group(0)
                    # Look for any entry with this date pattern
                    for k, v in metadata_map.items():
                        if date_pattern in k:
                            metadata_entry = v
                            matched_key = k
                            break
                    if metadata_entry:
                        break
        
        # Special case for PSR format with hyphen
        if not metadata_entry and 'PSR' in base_name:
            # Extract year, month, day if present
            psr_match = re.search(r'PSR (\d{4})-(\d{1,2})(?:-(\d{1,2}))?', base_name)
            if psr_match:
                year = psr_match.group(1)
                month = psr_match.group(2).zfill(2)  # Ensure 2 digits
                day = psr_match.group(3).zfill(2) if psr_match.group(3) else None
                
                # Try to match with PSC_YYYY_MM_DD format
                if day:
                    psc_key = f"PSC_{year}_{month}_{day}"
                else:
                    psc_key = f"PSC_{year}_{month}"
                
                if psc_key in metadata_map:
                    metadata_entry = metadata_map[psc_key]
                    matched_key = psc_key
        
        # Store metadata if found
        if metadata_entry:
            source_file_metadata[source_file] = metadata_entry
            matched_files.add(source_file)
        else:
            unmatched_files.add(source_file)
    
    # Third pass: update all chunks with matched metadata
    chunks_list = data.get("chunks", [])
    
    for source_file, metadata_entry in source_file_metadata.items():
        # Get all chunk indices for this source file
        chunk_indices = source_file_to_chunks.get(source_file, [])
        
        for idx in chunk_indices:
            chunk = chunks_list[idx]
            metadata = chunk.get("metadata", {})
            
            # Only update if title is empty or not set
            if not metadata.get("title") or metadata.get("title") == "":
                metadata["title"] = metadata_entry["title"]
            
            # Only update subjects if empty
            if not metadata.get("subjects") or metadata.get("subjects") == []:
                metadata["subjects"] = metadata_entry["subjects"]
            
            updated_count += 1
    
    # Save the updated JSON back to the same file
    with open(json_file_path, 'w', encoding='utf-8') as f:
        json.dump(data, f, ensure_ascii=False, indent=2)
    
    print(f"Updated {updated_count}/{total_chunks} chunks in {json_file_path}")
    print(f"Matched {len(matched_files)} unique files, failed to match {len(unmatched_files)} files")
    
    # Print some examples of unmatched files for debugging
    if unmatched_files:
        print("Examples of unmatched files:")
        for file in list(unmatched_files)[:5]:
            print(f"  - {file}")
    
    return updated_count

def update_all_years(consolidated_dir, metadata_files):
    """
    Update all year JSON files with metadata.
    
    Args:
        consolidated_dir (str): Directory containing consolidated JSON files
        metadata_files (list): List of paths to metadata files
        
    Returns:
        dict: Summary of updates by year
    """
    # Load all metadata
    metadata_map = load_metadata_from_excel_csv(metadata_files)
    
    # Get all consolidated JSON files
    json_pattern = os.path.join(consolidated_dir, "psc_*_all_chunks.json")
    json_files = glob.glob(json_pattern)
    
    results = {}
    
    for json_file in json_files:
        # Extract year from filename
        year_match = re.search(r'psc_(\d{4})_all_chunks', os.path.basename(json_file))
        if year_match:
            year = year_match.group(1)
            
            # Update the file
            updated_count = update_json_with_metadata(json_file, metadata_map)
            
            results[year] = {
                "file": os.path.basename(json_file),
                "updated_chunks": updated_count
            }
    
    return results

# Cell 6: Explore the metadata files
def explore_metadata_file(file_path):
    """View a sample of the metadata file to confirm structure"""
    try:
        if file_path.endswith('.xlsx') or file_path.endswith('.xls'):
            try:
                df = pd.read_excel(file_path)
            except ImportError as e:
                print(f"Error: {e}")
                print("Trying to read CSV instead...")
                # Check if CSV version exists
                csv_version = file_path.rsplit('.', 1)[0] + '.csv'
                if os.path.exists(csv_version):
                    df = pd.read_csv(csv_version)
                else:
                    print(f"CSV version not found: {csv_version}")
                    return None
        elif file_path.endswith('.csv'):
            # Detect encoding
            encoding = detect_encoding(file_path)
            df = pd.read_csv(file_path, encoding=encoding)
        else:
            print(f"Unsupported file format: {file_path}")
            return None
        
        print(f"File: {file_path}")
        print(f"Total rows: {len(df)}")
        print(f"Columns: {df.columns.tolist()}")
        print("\nSample data (first 5 rows):")
        return df.head()
    except Exception as e:
        print(f"Error exploring file {file_path}: {e}")
        return None

# Run this to explore a metadata file
# Uncomment to explore your files
# if METADATA_FILES:
#     explore_metadata_file(METADATA_FILES[0])

# Cell 7: Update a specific year
# For updating a single year (adjust the year as needed)
def update_specific_year(year, metadata_files, consolidated_dir):
    metadata_map = load_metadata_from_excel_csv(metadata_files)
    json_file_path = os.path.join(consolidated_dir, f"psc_{year}_all_chunks.json")
    
    if os.path.exists(json_file_path):
        updated_count = update_json_with_metadata(json_file_path, metadata_map)
        print(f"\nUpdated {updated_count} chunks in the {year} file")
    else:
        print(f"\nFile not found: {json_file_path}")

# Cell 8: Main execution - run this to process a specific year
# Replace with the year you want to process
# YEAR_TO_PROCESS = "1974"

# print(f"Processing year: {YEAR_TO_PROCESS}")
# update_specific_year(YEAR_TO_PROCESS, METADATA_FILES, CONSOLIDATED_DIR)

# Cell 9: Update all years at once
# Run this to update all years
def update_all_years_with_summary():
    results = update_all_years(CONSOLIDATED_DIR, METADATA_FILES)
    
    print("\nUpdate results by year:")
    
    # Create a DataFrame for better display
    summary_data = []
    for year, result in sorted(results.items()):
        summary_data.append({
            "Year": year,
            "File": result['file'],
            "Updated Chunks": result['updated_chunks']
        })
    
    # Try to create a DataFrame, but handle the case where pandas might be limited
    try:
        return pd.DataFrame(summary_data)
    except:
        # Print in tabular format if DataFrame creation fails
        print("\nYear | File | Updated Chunks")
        print("-" * 40)
        for item in summary_data:
            print(f"{item['Year']} | {item['File']} | {item['Updated Chunks']}")
        return summary_data

# Uncomment this line to update all years
update_all_years_with_summary()

✓ pandas is already installed
✓ openpyxl is already installed
Loading metadata from: D:\Technical_projects\PSAI\raw_data\PSC\000_LIMBFILES\1970s.xlsx
Columns in file: ['Category', 'media relative path', 'file name', 'title', 'subject', 'creator', 'date', 'publication', 'collection', 'series', 'physical location', 'notes', 'format', 'clearance']
Loading metadata from: D:\Technical_projects\PSAI\raw_data\PSC\000_LIMBFILES\80s.xlsx
Columns in file: ['Category', 'media relative path', 'file name', 'title', 'subject', 'creator', 'date', 'publication', 'collection', 'series', 'physical location', 'notes', 'format', 'clearance']
Loading metadata from: D:\Technical_projects\PSAI\raw_data\PSC\000_LIMBFILES\90s.csv
Detected encoding for 90s.csv: ascii (confidence: 1.00)
Columns in file: ['Category', 'media relative path', 'file name', 'title', 'subject', 'creator', 'date', 'publication', 'collection', 'series', 'physical location', 'notes', 'format', 'clearance']
Loading metadata from: D:\Techni

,Year,File,Updated Chunks
0,1973,psc_1973_all_chunks.json,0
1,1974,psc_1974_all_chunks.json,1043
2,1975,psc_1975_all_chunks.json,587
3,1976,psc_1976_all_chunks.json,623
4,1977,psc_1977_all_chunks.json,671
5,1978,psc_1978_all_chunks.json,736
6,1979,psc_1979_all_chunks.json,764
7,1980,psc_1980_all_chunks.json,817
8,1981,psc_1981_all_chunks.json,829
9,1982,psc_1982_all_chunks.json,843
